In [ ]:
# Find the base for various number of mixture signals. Add the avlternative vase features sequentially and select them according to the approximation to the external error.

In [1]:
import numpy as np
from functions import *
from dataload import load_data, get_clusters, gen_base, next_sample_dset
from experiment import FeatureSelection

# Load data and data basis
iqdata, iqnoise = load_data()
dbasis = get_clusters()

# Create a data generator and the main class instance
# The indices of base centroids in iqdata
idx_basis = list(dbasis.keys())
Abase = iqdata[idx_basis].transpose()

# How to use the data:

# Generate new dset with randomly mixed signals as y plus a noise of desired level
class_sizes = [0, 0, 0, 0, 0, 100]  # Set sample size for i-th collision
# dset = gen_base(iqdata, iqnoise, dbasis, cls_sizes)
# print(list(dset[0].keys())) # List of the keys in each data sample

# To get the data in easy way while setting noise, use the function
def get_dset(noise_lvl):
    # The arguments of the called function is in the cell visibility
    # We suppose they do not change during the experiment
    dset = gen_base(iqdata, iqnoise, dbasis, cls_sizes=class_sizes, noise_level=noise_lvl)
    return dset

# This instance delivers the new y with answers
# next_sample = next_sample_dset(dset, idx_basis) # Use after generation new dset

# This instance carry models for each y
# fs = FeatureSelection(Abase) # Use after get a new y

In [2]:
# The parameters of the computational experiment
n_models = 6
n_classes = len(class_sizes)
# Later we check the reconstructed signal with one of the cluster's signals

In [3]:
# Generate the new noisy dset of mixtures and create a datasample generator
noise_coeff = 1.0
db = get_dset(noise_coeff)
next_sample = next_sample_dset(db, idx_basis)

In [4]:
# Check the reconstruction quality
cnt_err = 0
cnt_resolved = 0
X_cls = np.empty([0,n_models * n_classes])  # objects to 4 class classification
X_cls_haus = X_cls.copy()

y_cls = []  # target 4 classes
for i in range(100):
    # Get a sample from the dataset
    answer_y, answer_X, answer_A, answer_coeff, answer_shift = next(next_sample)
    fs = FeatureSelection(Abase, max_models = n_models ) # Reset the list of models
    fs.MAX_MODELS = n_models
    fs.MAX_BASIS = 1

    # Run the reconstruction procedure
    # fs.mdl = run(fs.A, answer_y, fs.mdl, max_basis=5, max_models=6)
    i_dist = np.array([])
    i_haus = i_dist.copy()
    # For each number of collided signals
    for c in range(n_classes):
        fs.run(answer_y)
        i_dist = np.hstack((i_dist, np.array([values['err'] for values in fs.mdl.values()])))
        # Use alternative error, too
        fs.update_error(answer_y)
        i_haus = np.hstack((i_dist, np.array([values['err'] for values in fs.mdl.values()])))
        # print(np.shape(i_dist))
    # Make the dataset for classification
    X_cls = np.vstack((X_cls, i_dist.transpose()))
    X_cls_haus = np.vstack((X_cls, i_haus.transpose()))
    y_cls.append(len(answer_A))
    # construct

    # Plot the best model
    # fs.plot_mdl(answer_y, 1)
    # Check the quality of the reconstruction by comparing with the answer
    best_A = fs.best_model()['fea']
    dist = fs.best_model()['err']
    if set(answer_A) == set(best_A):
        print(i, 'errors:', dist)
    else:
        cnt_err += 1
        print(i, 'errors:', dist, 'answer:', answer_A, 'model:', best_A)
    len_resolved = len(set(answer_A) & set(best_A))
    if len_resolved > 0:
        cnt_resolved += 1
        print(i, 'resolved:', len_resolved)
print('_____________________')
print(cnt_err , cnt_resolved)



ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 36 and the array at index 1 has size 42